In [1]:
#从transfer_date列筛选年份为2000-2024

import pandas as pd

df = pd.read_csv(".\\transfermarket_data\\transfers.csv")  # 原始表

date_col = "transfer_date"

# 解析日期 -> 年
df["year"] = pd.to_datetime(df[date_col].astype(str), errors="coerce").dt.year

# 过滤 2000..2024（含边界）
out = df[df["year"].between(2000, 2024, inclusive="both")].copy()

# 保存
out.to_csv("transfers_2000_2024.csv", index=False, encoding="utf-8-sig")


In [ ]:
# club_transfer_graph.py
# -*- coding: utf-8 -*-
"""
按步骤构建“俱乐部为节点、人数为权重的有向图”，并导出边表、GEXF 和示意图。

步骤目录：
0) 导入 & 参数
1) 读取 CSV
2) 自动识别关键列
3) 可选的时间过滤（year 或 transfer_date）
4) 选取并清洗 (from_club, to_club, player_key)
5) 统计加权有向边（weight & unique_players）
6) 构建 NetworkX DiGraph & 计算强度
7) 导出：边表 CSV、GEXF
8) 可视化（选取活跃节点 + 较粗的边）
9) 打印/保存 Top 榜单
"""

# --------------------------- 0) 导入 & 参数 ---------------------------
import argparse
import math
from pathlib import Path

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import unicodedata


def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--csv", required=True, help="转会记录 CSV 路径")
    p.add_argument("--out_dir", default="out_graph", help="输出目录")
    # 过滤/取数参数
    p.add_argument("--year_min", type=int, default=None, help="最小年份（包含）")
    p.add_argument("--year_max", type=int, default=None, help="最大年份（包含）")
    p.add_argument("--remove_without_club", action="store_true", help="过滤名称为 'Without Club' 的记录")
    p.add_argument("--weight_mode", choices=["events", "unique"], default="events",
                   help="权重定义：events=事件数；unique=唯一球员数")
    # 可视化参数（子图抽样）
    p.add_argument("--top_k", type=int, default=40, help="总强度最高的前 K 个节点用于画图")
    p.add_argument("--edge_quantile", type=float, default=0.60, help="边权重分位数阈值（0~1）用于筛边")
    p.add_argument("--label_top_n", type=int, default=15, help="给强度前 N 的节点打标签")
    return p.parse_args()


# --------------------------- 1) 读取 CSV ---------------------------
def read_csv(csv_path: Path) -> pd.DataFrame:
    try:
        return pd.read_csv(csv_path, low_memory=False)
    except UnicodeDecodeError:
        return pd.read_csv(csv_path, low_memory=False, encoding="latin1")


# --------------------------- 2) 自动识别关键列 ---------------------------
def normalize_col(col: str) -> str:
    col = unicodedata.normalize("NFKC", str(col)).strip().lower()
    safe = []
    for ch in col:
        safe.append(ch if ch.isalnum() else "_")
    return "".join(safe)


def pick_col(cols_norm_map, candidates):
    for raw, norm in cols_norm_map.items():
        for cand in candidates:
            # 允许轻微噪声：精确 / 结尾匹配 / 包含
            if norm == cand or norm.endswith("_" + cand) or cand in norm:
                return raw
    return None


def detect_columns(df: pd.DataFrame):
    cols_norm = {c: normalize_col(c) for c in df.columns}

    from_candidates = [
        "from_club","club_from","fromteam","fromclub","from","source_club","source",
        "selling_club","seller_club","prev_club","club_name_from","from_team",
        "from_club_id","from_club_name"
    ]
    to_candidates = [
        "to_club","club_to","toteam","toclub","to","target_club","target",
        "buying_club","buyer_club","next_club","club_name_to","to_team",
        "to_club_id","to_club_name"
    ]
    player_id_candidates = ["player_id","id","tm_player_id","sofifa_id","person_id","player"]
    player_name_candidates = ["player_name","name","player_full_name"]
    date_candidates = ["year","transfer_date","date","transfered","transferred","move_date","deal_date","transfer_season","season"]

    col_from = pick_col(cols_norm, from_candidates)
    col_to   = pick_col(cols_norm, to_candidates)
    col_pid  = pick_col(cols_norm, player_id_candidates) or pick_col(cols_norm, player_name_candidates)
    col_date = pick_col(cols_norm, date_candidates)

    # 若同时存在 name 和 id，优先 id 作为 player_key
    raw_cols = df.columns
    if "player_id" in raw_cols:
        col_pid = "player_id"
    if "from_club_name" in raw_cols:
        col_from = "from_club_name"
    if "to_club_name" in raw_cols:
        col_to = "to_club_name"

    return dict(from_col=col_from, to_col=col_to, player_key_col=col_pid, date_col=col_date)


# --------------------------- 3) 可选的时间过滤 ---------------------------
def filter_by_time(df: pd.DataFrame, col_date: str, year_min: int, year_max: int) -> pd.DataFrame:
    if year_min is None and year_max is None:
        return df

    df2 = df.copy()
    # 优先 year 列；没有则从 transfer_date 解析
    if "year" in df2.columns:
        df2["__year__"] = pd.to_numeric(df2["year"], errors="coerce")
    elif col_date and col_date in df2.columns:
        dt = pd.to_datetime(df2[col_date], errors="coerce")
        df2["__year__"] = dt.dt.year
    else:
        df2["__year__"] = np.nan  # 无法过滤

    if year_min is not None:
        df2 = df2[df2["__year__"] >= year_min]
    if year_max is not None:
        df2 = df2[df2["__year__"] <= year_max]
    df2 = df2.drop(columns=["__year__"])
    return df2


# --------------------------- 4) 选取并清洗列 ---------------------------
def build_work_df(df: pd.DataFrame, cols: dict, remove_without_club: bool) -> pd.DataFrame:
    for k in ["from_col","to_col","player_key_col"]:
        if not cols.get(k):
            raise ValueError(f"无法识别列：{k}，请检查 CSV 列名。")

    work = df[[cols["from_col"], cols["to_col"], cols["player_key_col"]]].copy()
    work.rename(columns={
        cols["from_col"]: "from_club",
        cols["to_col"]: "to_club",
        cols["player_key_col"]: "player_key"
    }, inplace=True)

    # 丢缺失 & 空值
    work = work.dropna(subset=["from_club","to_club","player_key"])
    work = work[(work["from_club"].astype(str).str.len() > 0) &
                (work["to_club"].astype(str).str.len() > 0)]

    # 丢自环
    work = work[work["from_club"] != work["to_club"]]

    # 可选：过滤 Without Club
    if remove_without_club:
        work = work[(work["from_club"] != "Without Club") & (work["to_club"] != "Without Club")]

    return work


# --------------------------- 5) 统计加权有向边 ---------------------------
def aggregate_edges(work: pd.DataFrame, weight_mode: str) -> pd.DataFrame:
    if weight_mode == "events":
        agg_weight = ("player_key", "size")  # 事件数
    else:
        agg_weight = ("player_key", pd.Series.nunique)  # 唯一球员数

    edge_counts = (
        work.groupby(["from_club","to_club"])
            .agg(weight=agg_weight,
                 unique_players=("player_key", pd.Series.nunique))
            .reset_index()
            .sort_values("weight", ascending=False)
    )
    return edge_counts


# --------------------------- 6) 构图 & 强度 ---------------------------
def build_graph(edge_counts: pd.DataFrame) -> tuple[nx.DiGraph, dict, dict, dict]:
    G = nx.DiGraph()
    for _, r in edge_counts.iterrows():
        G.add_edge(r["from_club"], r["to_club"],
                   weight=int(r["weight"]),
                   unique_players=int(r["unique_players"]))
    out_strength = dict(G.out_degree(weight="weight"))
    in_strength  = dict(G.in_degree(weight="weight"))
    total_strength = {n: out_strength.get(n, 0) + in_strength.get(n, 0) for n in G.nodes()}
    return G, out_strength, in_strength, total_strength


# --------------------------- 7) 导出 CSV & GEXF ---------------------------
def export_artifacts(edge_counts: pd.DataFrame, G: nx.DiGraph, out_dir: Path):
    out_dir.mkdir(parents=True, exist_ok=True)
    edges_csv = out_dir / "global_transfers_edges.csv"
    gexf_path = out_dir / "club_transfers_graph.gexf"

    edge_counts.to_csv(edges_csv, index=False)
    nx.write_gexf(G, gexf_path)
    return edges_csv, gexf_path


# --------------------------- 8) 可视化（活跃子图） ---------------------------
def visualize_graph(G: nx.DiGraph, total_strength: dict,
                    out_dir: Path, top_k=40, edge_quantile=0.60, label_top_n=15):
    # 选择 Top-K 节点
    nodes_sorted = sorted(total_strength.items(), key=lambda x: x[1], reverse=True)
    top_nodes = [n for n, _ in nodes_sorted[:min(top_k, len(nodes_sorted))]]
    H = G.subgraph(top_nodes).copy()

    # 筛边：按权重分位数与 3 取 max
    if H.number_of_edges() > 0:
        weights = [d["weight"] for _, _, d in H.edges(data=True)]
        w_thr = max(3, int(np.quantile(weights, edge_quantile))) if len(weights) else 1
        strong_edges = [(u, v) for u, v, d in H.edges(data=True) if d.get("weight", 1) >= w_thr]
        H = H.edge_subgraph(strong_edges).copy()

    if H.number_of_nodes() == 0:
        print("筛选后子图为空，跳过可视化。")
        return None

    # 布局
    k = 0.6 / math.sqrt(max(1, H.number_of_nodes()))
    pos = nx.spring_layout(H, seed=42, k=k)

    # 节点大小 & 标签
    node_sizes = [80 + 2.0 * total_strength.get(n, 0) for n in H.nodes()]
    labels = {}
    for n, _s in sorted({n: total_strength.get(n, 0) for n in H.nodes()}.items(),
                        key=lambda x: x[1], reverse=True)[:label_top_n]:
        labels[n] = n

    # 边宽度归一
    allw = [d["weight"] for _, _, d in H.edges(data=True)]
    if allw:
        w_min, w_max = min(allw), max(allw)
        denom = (w_max - w_min) if (w_max - w_min) > 0 else 1
        edge_widths = [1.0 + 3.0 * (d["weight"] - w_min) / denom for _, _, d in H.edges(data=True)]
    else:
        edge_widths = [1.0] * H.number_of_edges()

    # 画图
    plt.figure(figsize=(12, 10))
    nx.draw_networkx_nodes(H, pos, node_size=node_sizes)
    nx.draw_networkx_edges(H, pos, width=edge_widths, arrows=True, arrowstyle='-|>', arrowsize=10)
    nx.draw_networkx_labels(H, pos, labels=labels, font_size=9)
    plt.axis("off")
    out_png = out_dir / "club_transfers_network.png"
    plt.tight_layout()
    plt.savefig(out_png, dpi=180, bbox_inches="tight")
    plt.close()
    return out_png


# --------------------------- 9) Top 榜单 ---------------------------
def export_tops(edge_counts: pd.DataFrame,
                out_strength: dict, in_strength: dict,
                out_dir: Path, k=20):
    top_edges = edge_counts.head(25)
    top_out = pd.DataFrame(sorted(out_strength.items(), key=lambda x: x[1], reverse=True)[:k],
